In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys

aml_dl_path = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..', ''))
sys.path.append(aml_dl_path)

import cnn_pose_estimation.model.tf_model as tf_model
from cnn_pose_estimation.training.config import network_params

import numpy as np
import tensorflow as tf

# Fixing seed
np.random.seed(seed=42)

def fakeImageInput(network_params):
    random_image = np.random.randn(network_params['image_height'],network_params['image_width'],network_params['image_channels'])
    image = np.transpose(random_image,(2,1,0)) # If the image is WxHxC, make it CxWxH
    image = image.flatten()

    return image, random_image

nn = tf_model.multi_modal_network_fp(dim_input=network_params['image_size']+32,
                                      network_config=network_params)

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

fc_op = nn['fc_pose_out']
features_op = nn['features']
loss_op = nn['loss_pose']
input_tensor = nn['nn_input']
position = nn['pose']
train_op = tf_model.train_adam_step(loss_op)

init_fn = nn['init_fn']



STATE IDX: 31
IMAGE IDX: 150559
SHAPE INPUT: (?, 150560)
SHAPE STATE: (?, 32)
SHAPE IMAGE: (?, 150528)
SHAPE: (?, 224, 224, 3)


In [3]:
# Initialialise session and variables

init_op = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init_op)
init_fn(sess)

# Run a simple test with a fake image
image, random_image = fakeImageInput(network_params)

### Training ###

p0 = np.array([0,0,0])
p1 = np.array([0,1,0])
p2 = np.array([0,1,1])
points = np.concatenate((p0,p1,p2))

for iteration in range(1000):
    with tf.device("/gpu:0"):
        loss = sess.run([loss_op,train_op],feed_dict={input_tensor: np.expand_dims(np.r_[np.zeros(32),image],axis=0),
                                     position: np.expand_dims(points,axis=0)
                                    })

        # print every 50 iters
        if iteration%100 == 0:
            print "iteration %d loss: "%(iteration), loss


######## SAVING MODEL #######
save_path = saver.save(sess, "model.ckpt")
print("Model saved in file: %s" % save_path)
    

sess.close()

iteration 0 loss:  [0.060000017, None]
iteration 100 loss:  [0.058676071, None]
iteration 200 loss:  [0.056601781, None]
iteration 300 loss:  [0.05064946, None]
iteration 400 loss:  [0.038728632, None]
iteration 500 loss:  [0.024177203, None]
iteration 600 loss:  [0.011285049, None]
iteration 700 loss:  [0.0034767813, None]
iteration 800 loss:  [0.00067143812, None]
iteration 900 loss:  [8.7477878e-05, None]
Model saved in file: model.ckpt
